In [ ]:
# https://github.com/Jesse989/game-oracle
# https://towardsdatascience.com/creating-a-dataset-from-scratch-b8e2f8752436
# https://rawg.io
# https://api.rawg.io/docs/?format=openapi
# https://rawg.io/apidocs
MY_API_KEY = '39cf6ce7746d42cfb023c27a0ba708f9'   # il faut la créer --> 20000 requetes / mois

# en deux parties
# enregistrement de tous les jsons par search par nom en filtrant par plateforme avec 40 pages sizs
# traitement des json

# https://api.rawg.io/api/games?key=39cf6ce7746d42cfb023c27a0ba708f9&dates=2019-09-01,2019-09-30&page_size=1000&search=

In [ ]:
import requests 
import json
from time import sleep
from tqdm.auto import tqdm
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
from os import listdir

url = 'https://api.rawg.io/api/games?key=39cf6ce7746d42cfb023c27a0ba708f9&page_size=40&search='

In [ ]:
# STEP 1: JSON generation form vgsales_at.csv
vgchartz = pd.read_csv("vgsales_at.csv", sep=",")
print (vgchartz.shape)
vgchartz.head()

In [ ]:
list_jeux_df = vgchartz.Name.unique()
print (len(list_jeux_df))
list_jeux_df[:10]

In [ ]:
# methode de recup et enregistrement des données
def rawg_request(name):
    r = requests.get(url+name)
    return r

def save_game(res_get):
    if res_get.content:
        result = json.loads(res_get.content)
        if len(result['results']):
            file_name = result['results'][0]['slug']
            with open('rawg/' + file_name + '.json', 'w') as outfile:
                json.dump(result, outfile)

In [ ]:
# STEP 1: JSON generation fom 
for j in tqdm(list_jeux_df[10:]):
    res = rawg_request(j)
    save_game(res)
    sleep(0.5)

In [ ]:
# STEP 2: JSON --> Dataset
game_slug = []
game_name = []
game_playtime = []
game_id = []
game_metacritic = []
game_platforms = []
game_platforms_count = []
game_rating = []
game_ratings_count = []
game_reviews_text_count = []
game_score = []
game_esrb_rating = []
game_reviews_count = []
game_genres = []
game_released = []
game_suggestions_count = []

for f in tqdm(listdir('rawg')):
    
    with open('rawg/' + f) as json_file:
        data_json = json.load(json_file)
        
        game = data_json['results'][0]
        
        game_slug.append(game['slug'])
        game_name.append(game['name'])
        game_playtime.append(game['playtime'])
        game_id.append(game['id'])
        game_metacritic.append(game['metacritic'])
        if game['platforms']:
            game_platforms.append(','.join([p['platform']['name'].lower() for p in game['platforms']]))
            game_platforms_count.append(len(game['platforms']))
        else:
            game_platforms.append(np.nan)
            game_platforms_count.append(np.nan)

        game_rating.append(game['rating'])
        game_ratings_count.append(game['ratings_count'])
        game_reviews_text_count.append(game['reviews_text_count'])
        game_score.append(game['score'])
        game_esrb_rating.append(game['esrb_rating']['name'] if game['esrb_rating'] else np.nan)
        game_reviews_count.append(game['reviews_count'])
        game_genres.append(','.join([g['name'].lower() for g in game['genres']]))
        game_released.append(game['released'])
        game_suggestions_count.append(game['suggestions_count'])

rawg = pd.DataFrame({'Name': game_name,
                     'Slug' : game_slug,
                     'Playtime' : game_playtime,
                     'Metacritic' : game_metacritic,
                     'Platforms' : game_platforms,
                     'Platforms count' :  game_platforms_count,
                     'Rating' : game_rating,
                     'Rating count' : game_ratings_count,
                     'Reviews text count' : game_reviews_text_count,
                     'Score' : game_score,
                     'ESRD Rating' : game_esrb_rating,
                     'Reviews count' : game_reviews_count,
                     'Genres' : game_genres,
                     'Released' : game_released,
                     'Rawg_id'  : game_id,
                     'Suggestions count' : game_suggestions_count,
                    })

rawg.head()

In [ ]:
gg = rawg['Genres'].str.get_dummies(sep=',')
pp = rawg['Platforms'].str.get_dummies(sep=',')
gg_pp = pd.concat([gg, pp], axis=1)
gg_pp.head()

In [ ]:
rawg.to_csv("rawg_sl.csv", sep=",", encoding='utf-8', index=False)